<a href="https://colab.research.google.com/github/edsondamasceno/classification-COVID-19/blob/main/XGBoost_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import os
import cv2
import xgboost
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
path = '/content/drive/MyDrive/COVID-19/CT/Pre-trained/Test_CNP_mobilenetv2.csv'

In [ ]:
TEST = 0.2
TRAIN = 1 - TEST

#file_data = open(path_arquivo_descritor, 'r')
file_data = pd.read_csv(path, sep=',')

FEATURES = file_data.drop('Labels', axis=1)
LABELS = file_data['Labels'].astype('int')

# formata os dados de terino e test_size
X_train, X_test, y_train, y_test = train_test_split(FEATURES, LABELS, test_size=TEST)

train_x = X_train
train_y = y_train
test_x = X_test
test_y = y_test

dataset = xgboost.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'num_class':3,'objective':'multi:softmax' }
model_xg = xgboost.train(params, dataset, num_boost_round=150, evals=watchlist)

test_x = xgboost.DMatrix(test_x)
result = model_xg.predict(test_x)

In [ ]:
recall = recall_score(test_y, result, average='macro')
precision = precision_score(test_y, result, average='macro')
f1 = f1_score(test_y, result, average='macro')
kappa = cohen_kappa_score(test_y, result)
acc = accuracy_score(test_y, result)
auc_score = roc_auc_score(test_y, result, average="macro")

print("============================ Result ============================")
print("Accuracy:", (acc*100))
print("Precision:", (precision*100))
print("Recall:", (recall*100))
print("F1-score:", f1*100)
print("AUC:", auc_score)
print("Kappa:", kappa)
print("================================================================")

In [ ]:
print(metrics.confusion_matrix(test_y, result))